# PA4: Implementing linear classifiers
### Authors: David Laessker, Peter Fagrell 



## Exercise Question

Since the perceptron is a linnear classifier it can only classify data that is linearly seperable. This means that it can only classify data that can be seperated by a line. We can represent this very arcaheicly by the following:

Trainig data 1 would look like this 

x|x
-|-
x|o

and training data 2 would look like this

o|x
-|-
x|o

The top two boxes are Gothenburg/Sydney bottom two are Paris,  the right boxes are July and the left boxes are December.


As we can see we can easily draw a line that separates the two classes in example 1 but not in example 2. This means that the perceptron can only classify example 1 and not example 2.

--------------------------------
## Implementing the SVC

### The following table shows the accuracy achived with the different classifiers

| Classifier | Accuracy |
| --- | --- |
| PegasosSVC | 0.8431 |
| PegasosLREG | 0.8309 |

# Our implemetation:

In [1]:
#Imports for lab 4
import numpy as np
from aml_perceptron import LinearClassifier

import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


#### The _PegasosSVC_ was implemented with by translating the pseudocode from the document _'Clarification of the pseudocode in the Pegasos paper'_ with the help of the clarification in said document. The algorithm was run with several different combinations of parameters before settling on the following:

| Parameter | Value |
| --- | --- |
| lambda_reg | 0.01 |
|n_iter | 100 000 |

### The code cell below shows the implementation of the PegasosSVC algorithm:

In [2]:
class PegasosSVC(LinearClassifier):
    """
    Implementation of the Pegasos algorithm for SVCs.
    """

    def __init__(self, lambda_reg=0.1, n_iter=1000000):
        self.lambda_reg = lambda_reg
        self.n_iter = n_iter

    def fit(self, X, Y):

        # Preprocess the data
        self.find_classes(Y)
        Y_encoded = self.encode_outputs(Y)

        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weights
        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        self.lambda_reg = 1/n_features

        # Pegasos algorithm implemented
        # like the peudocode in the paper
        for t in range(1, self.n_iter):
            rand = np.random.randint(0, len(X))
            x, y = X[rand], Y_encoded[rand]

            n = 1/(self.lambda_reg*t)

            score = x.dot(self.w)

            if y*score <= 1:
                self.w = (1 - n*self.lambda_reg) * self.w + n*y*x
            else:
                self.w = (1 - n*self.lambda_reg) * self.w



#### The following two code cells is doc_classification.py that was provided by the course. It was modified to fit the _PegasosSVC_ algorithm instead of perceptron as it was orignally. We split it up and simplified it to make it easier to understand and to make it easier to implement the _PegasosSVC_ and _PegasosLREG_ algorithms.

In [3]:
# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels.


def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y


#### When we run the code cell below it will print the **accuracy** of the _PegasosSVC_ algorithm.

## Accuracy: 0.8431

In [4]:
# Read all the documents.
X, Y = read_data(
    'data/all_sentiment_shuffled.txt')
# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=69)

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    # NB that this is our Perceptron, not sklearn.linear_model.Perceptron
    PegasosSVC()
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 10.66 sec.
Accuracy: 0.8431.


#### PegsosLREG is the 

| Parameter | Value |
| --- | --- |
| lambda_reg | 0.1 |
|n_iter | 100 000 |

### The code cell below shows the implementation of the PegasosLREG algorithm:

In [5]:
class PegasosLREG(LinearClassifier):
    """
    Implementation of the Pegasos algorithm for logistic regression.
    """

    def __init__(self, lambda_reg=0.1, n_iter=100000):
        self.lambda_reg = lambda_reg
        self.n_iter = n_iter

    def fit(self, X, Y):

        # Preprocess the data
        self.find_classes(Y)
        Y_encoded = self.encode_outputs(Y)

        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weights
        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        self.lambda_reg = 1/n_features

        # Pegasos algorithm implemented with logistic regression
        for t in range(1, self.n_iter):
            rand = np.random.randint(0, len(X))
            x, y = X[rand], Y_encoded[rand]

            n = 1/(self.lambda_reg*t)

            score = x.dot(self.w)
            gradient = -y * x * (1 - 1/(1 + np.exp(-y * score)))

            self.w = (1 - n * self.lambda_reg) * self.w - n * gradient
            self.w *= min(1, 1 / (np.sqrt(self.lambda_reg)
                          * np.linalg.norm(self.w)))


In [8]:


class PegasosLR(LinearClassifier):
    """
    A straightforward implementation of the pegasos learning algorithm.
    """

    def __init__(self, n_iter=20, reg_lambda=0.001, print_loss=True):
        """
        The constructor can optionally take the parameters
        - n_iter the number of times to iterate through the training set
        - reg_lambda the regularization parameter
        """

        self.n_iter = int(n_iter)
        self.reg_lambda = reg_lambda
        self.print_loss = print_loss

        # Initialize the array for losses
        self.losses = []

        # to avoid warnings about using non-defined attributes
        self.w = None

    def fit(self, X, Y):
        """
        Train a linear classifier using the pegasos learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Pegasos algorithm:
        for i in range(self.n_iter):
            t = 1

            for x, y in zip(X, Ye):

                t += 1

                # calculate the learning rate, eta
                eta = 1 / (self.reg_lambda * t)

                # Compute the output score for this instance.
                score = x.dot(self.w)

                self.losses.append(np.log(1 + np.exp(-y * score)))

                # Update the weights by the log loss algorithm
                loss_gradient = y / (1 + np.exp(y * score))
                self.w = (1 - eta * self.reg_lambda) * \
                    self.w + loss_gradient * eta * x

            # if i % 2 == 0 and self.print_loss:
            #     avg_loss = sum(self.losses) / len(self.losses)
            #     print(f"Iteration {i}, average loss: {round(avg_loss, 4)}")


In [16]:
# Read all the documents.
X, Y = read_data(
    'data/all_sentiment_shuffled.txt')
# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=69)

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    # NB that this is our Perceptron, not sklearn.linear_model.Perceptron
    PegasosLR()
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))


Training time: 2.69 sec.
Accuracy: 0.8275.
